In [2]:
import pandas as pd
!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.sectorperformance import SectorPerformances
import matplotlib.pyplot as plt
import requests
import os
from time import sleep

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

# CPU version of pytorch has smaller footprint - see installation instructions in
# pytorch documentation - https://pytorch.org/get-started/locally/
!pip3 install torch==1.12+cpu torchvision==0.13.0+cpu torchtext==0.13.0 -f https://download.pytorch.org/whl/cpu/torch_stable.html

!pip3 install autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached setuptools-65.6.3-py3-none-any.whl (1.2 MB)
  Using cached wheel-0.38.4-py3-none-any.whl (36 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 60.10.0
    Uninstalling setuptools-60.10.0:
      Successfully uninstalled setuptools-60.10.0
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     ━

In [3]:
# API_KEY = os.getenv('API_KEY')
API_KEY = '9M60VHUB93B1UIWI'
symbol = 'BMW.FRK'
pcent_change_to_buy_on = 1.5
sleep_time = 14

In [164]:
# # get sector overview for stock
# url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={API_KEY}'
# r = requests.get(url)
# sector = r.json()['Sector']
# data = r.json()
# print(sector)
# print(data)

In [165]:
# # ***************Sentiment would be a good source but can't seem to get more than 50 items from the API so won't go back
# # far enough???????
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# # url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&apikey={API_KEY}
# url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbol}&time_from=20040825T0130&apikey={API_KEY}'
# print(url)
# r = requests.get(url)

# print(data)

In [166]:
# Get json object with the intraday data and another with  the call's metadata
# For the default date string index behavior
# ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')
# ts_data, ts_meta_data = ts.get_intraday(symbol, interval='1min', outputsize='full')


# ts_data['4. close'].plot()
# plt.title(f'Intraday Times Series for the {symbol} stock (1 min)')
# plt.show()

In [4]:
# Adjusted daily data
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='integer')
ts_data, ts_meta_data = ts.get_daily_adjusted(symbol, outputsize='full')


# ts_data['4. close'].plot()
# plt.title('Intraday Times Series for the GOOGL stock (1 min)')
# plt.show()

In [5]:
adjusted_data = ts_data
adjusted_data.tail()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
index,,,,,,,,,
4566,2005-01-07,34.60,34.72,34.30,34.57,21.4574,280363.0,0.0,1.0
4567,2005-01-06,34.55,34.90,34.53,34.70,21.5381,250466.0,0.0,1.0
4568,2005-01-05,34.20,34.70,34.15,34.65,21.5071,734946.0,0.0,1.0
4569,2005-01-04,33.70,34.48,33.67,34.39,21.3457,567720.0,0.0,1.0
4570,2005-01-03,33.30,33.80,33.30,33.75,20.9485,246820.0,0.0,1.0


In [6]:
# compute the percent change for each day
adjusted_data['day_pcent_change'] = 100 * ((ts_data['4. close'] - ts_data['1. open']) / ts_data['1. open'])


# ******need to predict whether price for tomorrow will go up by 2% or greater - so need label - buy - 0 or 1
# today's close data will be used to predict whether to buy tomorrow
# so model cannot have the most recent row in it, since that would be today's data
# from which tomorrow's prediction will be made

# shift all the next days data by one which will be used to determine whether it should have been bought or not
adjusted_data['next_days_pcent_change'] = adjusted_data['day_pcent_change'].shift(+1)

In [7]:
# create the label - binary classification of whether to buy or not based on whether the stock went up 2% or more
# that day

def buy_or_not(row):
    if row.next_days_pcent_change >= pcent_change_to_buy_on:
        return 1
    else:
        return 0

adjusted_data['buy_tomorrow'] = adjusted_data.apply(buy_or_not, axis='columns')

# next_days_pcent_change will pollute the data set so drop it

adjusted_data.drop(columns='next_days_pcent_change',
                  inplace=True)

In [8]:
# technical indicators - some of these are premium
ti = TechIndicators(key=API_KEY, output_format='pandas')
wma = [5, 10, 20, 40, 80]

# def concat_wma_df(df, period):
#     '''
#     concatenate the pandas dataframes being passed into the function
#     '''
    
wma_data = pd.DataFrame()

for period in wma:
    
    sleep(sleep_time) # avoid breaching the API request rate limit
    ti_data, ti_meta_data = ti.get_wma(symbol, interval='daily', time_period=period)
    wma_data[f'WMA_{period}'] = ti_data['WMA']

    
wma_data.describe() 

# *********get the resistance figures for the MAs?
# need to understand the relevance of the MAs better and what predictive power they are supposed to be happened so it can be represented properly




# ti_data.plot()
# plt.title('WMA indicator for  GOOGL stock (60 min)')
# plt.show()

,WMA_5,WMA_10,WMA_20,WMA_40,WMA_80
count,4567.000000,4562.000000,4552.000000,4532.000000,4492.000000
mean,51.433204,51.443447,51.464849,51.508324,51.599924
std,22.558871,22.533828,22.483755,22.381686,22.191139
min,12.127500,12.280200,12.566200,13.270200,13.574300
25%,27.764950,27.832625,27.804425,27.830400,27.980025
50%,55.902700,56.262750,56.873150,57.046050,56.546300
75%,70.579600,70.619900,70.569975,70.590000,70.584525
max,98.884000,98.297600,96.555600,94.075400,92.770000


In [9]:
# tech indicator data is received in the reverse order to price data so reverse it
rev_wma_data = wma_data[::-1]
rev_wma_data.reset_index(inplace=True)

# rev_ti_data.drop(index=rev_ti_data.index[0], 
#         axis=0, 
#         inplace=True)
rev_wma_data.head()

,date,WMA_5,WMA_10,WMA_20,WMA_40,WMA_80
0,2022-12-30,83.6173,83.6222,83.6899,83.9181,81.7652
1,2022-12-29,83.7027,83.6471,83.7578,83.9223,81.6605
2,2022-12-28,83.5607,83.5398,83.7701,83.8882,81.5359
3,2022-12-27,83.7067,83.5889,83.8623,83.8883,81.4272
4,2022-12-23,83.6487,83.5449,83.8986,83.8549,81.3006


In [10]:
for period in wma:
    adjusted_data[f'WMA_{period}'] = rev_wma_data[f'WMA_{period}']
#     these should be percentage diff to close as well?
    adjusted_data[f'WMA_{period}_pcent_diff_to_close'] = (adjusted_data['4. close'] - adjusted_data[f'WMA_{period}']) / adjusted_data['4. close']
    adjusted_data[f'WMA_{period}_pcent_diff_to_open'] = (adjusted_data['1. open'] - adjusted_data[f'WMA_{period}']) / adjusted_data['4. close']
    adjusted_data.drop(columns=f'WMA_{period}', inplace=True)

In [174]:
adjusted_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,WMA_5_pcent_diff_to_close,WMA_5_pcent_diff_to_open,WMA_10_pcent_diff_to_close,WMA_10_pcent_diff_to_open,WMA_20_pcent_diff_to_close,WMA_20_pcent_diff_to_open,WMA_40_pcent_diff_to_close,WMA_40_pcent_diff_to_open,WMA_80_pcent_diff_to_close,WMA_80_pcent_diff_to_open
index,,,,,,,,,,,,,,,,,,,,,
0,2022-12-23,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,...,0.000016,-0.002854,0.001256,-0.001613,-0.002972,-0.005841,-0.002449,-0.005319,0.028086,0.025217
1,2022-12-22,84.20,84.20,83.01,83.29,83.29,443.0,0.0,1.0,-1.080760,...,-0.003650,0.007276,-0.003178,0.007748,-0.007997,0.002928,-0.006450,0.004476,0.025379,0.036304
2,2022-12-21,83.21,84.20,83.21,84.04,84.04,60.0,0.0,1.0,0.997476,...,0.004918,-0.004958,0.004976,-0.004900,-0.000143,-0.010019,0.002703,-0.007173,0.035488,0.025612
3,2022-12-20,82.17,83.75,82.17,83.75,83.75,742.0,0.0,1.0,1.922843,...,0.004433,-0.014432,0.002181,-0.016684,-0.003890,-0.022756,-0.000068,-0.018934,0.033823,0.014958
4,2022-12-19,82.66,83.86,82.66,83.70,83.70,281.0,0.0,1.0,1.258166,...,0.005305,-0.007121,0.001284,-0.011141,-0.005119,-0.017545,-0.000060,-0.012485,0.034876,0.022450


In [11]:
# ****************
# Now get bollinger band details - need to study this more and figure out what it means
# https://www.tradingview.com/support/solutions/43000501972-bollinger-bands-width-bbw/

# calculate the bollinger band width then look at it the difference over the last 6 months
# Bollinger Bands Width = (Upper Band - Lower Band) / Middle Band

# look for: The low volatility period is followed by a surge in volatility and 
# price breaks through the Upper Band or falls through the Lower Band signifying a 
# change in the sideways movement and the beginning of a new directional trend.


# In a Bullish BBW Squeeze

# BBW drops. (In the example below, the threshold is 6% however this changes from security to security and 
#             timeframe to timeframe)
# Price breaks through the Upper Band which starts a new upward trend. Volatility also increases.


# In a Bearish BBW Squeeze

# BBW drops. (In the example below, the threshold is 9% however this changes from security to 
#             security and timeframe to timeframe).
# Price falls below the Lower Band which starts a new downward trend. Volatility also increases.
        
bb_data, bb_meta_data = ti.get_bbands(symbol, interval='daily', time_period=20)

In [12]:
bb_data['bb_width'] = (bb_data['Real Upper Band'] - bb_data['Real Lower Band']) / bb_data['Real Middle Band']
bb_data.reset_index(inplace=True)
bb_data.head()

,date,Real Upper Band,Real Middle Band,Real Lower Band,bb_width
0,2022-12-30,85.7367,83.9810,82.2253,0.041812
1,2022-12-29,85.9181,84.0825,82.2469,0.043662
2,2022-12-28,86.1966,84.1800,82.1634,0.047912
3,2022-12-27,86.2291,84.2475,82.2659,0.047042
4,2022-12-23,86.2285,84.2410,82.2535,0.047186


In [13]:
adjusted_data['bb_width'] = bb_data['bb_width']
adjusted_data['bb_upper'] = bb_data['Real Upper Band']
adjusted_data['bb_lower'] = bb_data['Real Lower Band']
# add the below to percentages like day_pcent_change?
adjusted_data['pcent_diff_to_bb_upper'] = (adjusted_data['4. close'] - adjusted_data['bb_upper']) / adjusted_data['4. close']
adjusted_data['pcent_diff_to_bb_lower'] = (adjusted_data['4. close'] - adjusted_data['bb_lower']) / adjusted_data['4. close']

In [178]:
# # Don't seem to be able to get this with sufficient historical data
# sp = SectorPerformances(key=API_KEY, output_format='pandas')
# sp_data, sp_meta_data = sp.get_sector()
# sp_data.head()
# # sp_data['Rank A: Real-Time Performance'].plot(kind='bar')
# # plt.title('Real Time Performance (%) per Sector')
# # plt.tight_layout()
# # plt.grid()
# # plt.show()

In [14]:
sleep(sleep_time)
macd_data, macd_meta_data = ti.get_macd(symbol, series_type='close', interval='daily')
macd_data.reset_index(inplace=True)
adjusted_data['macd_signal'] = macd_data['MACD_Signal']
adjusted_data['macd_hist'] = macd_data['MACD_Hist']
adjusted_data['macd'] = macd_data['MACD']

In [15]:
sleep(sleep_time)
stochrsi_data, stochrsi_meta_data = ti.get_stochrsi(symbol, series_type='close', interval='daily', time_period=10, fastkperiod=6, fastdmatype=1)
stochrsi_data.reset_index(inplace=True)
adjusted_data['stochrsi_fast_k'] = stochrsi_data['FastK']
adjusted_data['stochrsi_fast_d'] = stochrsi_data['FastD']

In [16]:
sleep(sleep_time)
ppo_data, ppo_meta_data = ti.get_ppo(symbol, series_type='close', interval='daily', fastperiod=10, matype=1)
ppo_data.reset_index(inplace=True)
adjusted_data['ppo'] = ppo_data['PPO']

In [17]:
sleep(sleep_time)
aroo_data, aroo_meta_data = ti.get_aroon(symbol, interval='daily', time_period=14)
aroo_data.reset_index(inplace=True)
adjusted_data['aroon'] = aroo_data['Aroon Up'] - aroo_data['Aroon Down']

In [24]:
sleep(sleep_time)
minusdi_data, minusdi_meta_data = ti.get_minus_di(symbol, interval='daily', time_period=14)
minusdi_data.reset_index(inplace=True)
adjusted_data['minus_di'] = minusdi_data['MINUS_DI']

In [26]:
sleep(sleep_time)
plusdi_data, plusdi_meta_data = ti.get_plus_di(symbol, interval='daily', time_period=14)
plusdi_data.reset_index(inplace=True)
adjusted_data['plus_di'] = plusdi_data['PLUS_DI']

In [27]:
adjusted_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,pcent_diff_to_bb_lower,macd_signal,macd_hist,macd,stochrsi_fast_k,stochrsi_fast_d,ppo,aroon,minus_di,plus_di
index,,,,,,,,,,,,,,,,,,,,,
0,2022-12-30,83.60,83.60,83.33,83.37,83.37,10.0,0.0,1.0,-0.275120,...,0.013730,0.4011,-0.1868,0.2143,13.8567,41.2348,-1.4906,-35.7143,33.5343,15.5017
1,2022-12-29,83.48,84.06,83.12,84.05,84.05,1684.0,0.0,1.0,0.682798,...,0.021453,0.4478,-0.1845,0.2634,100.0000,68.6130,-1.2474,-35.7143,35.0363,14.2230
2,2022-12-28,83.50,83.50,83.21,83.28,83.28,1158.0,0.0,1.0,-0.263473,...,0.013408,0.4940,-0.2410,0.2529,0.0000,37.2260,-1.1117,-64.2857,32.4979,18.4488
3,2022-12-27,84.26,84.37,83.86,83.86,83.86,754.0,0.0,1.0,-0.474721,...,0.019009,0.5542,-0.2399,0.3143,81.1073,74.4519,-0.8960,-71.4286,33.1367,16.2160
4,2022-12-23,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,...,0.016695,0.6142,-0.2851,0.3291,72.6758,67.7966,-0.7531,-78.5714,35.4387,14.7623


In [19]:
# clean up NaNs
final_data = adjusted_data.dropna().reset_index(drop=True)


In [20]:
final_data.head()

,index,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,...,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,macd_hist,macd,stochrsi_fast_k,stochrsi_fast_d,ppo,aroon
0,2022-12-30,83.60,83.60,83.33,83.37,83.37,10.0,0.0,1.0,-0.275120,...,82.2253,-0.028388,0.013730,0.4011,-0.1868,0.2143,13.8567,41.2348,-1.4906,-35.7143
1,2022-12-29,83.48,84.06,83.12,84.05,84.05,1684.0,0.0,1.0,0.682798,...,82.2469,-0.022226,0.021453,0.4478,-0.1845,0.2634,100.0000,68.6130,-1.2474,-35.7143
2,2022-12-28,83.50,83.50,83.21,83.28,83.28,1158.0,0.0,1.0,-0.263473,...,82.1634,-0.035022,0.013408,0.4940,-0.2410,0.2529,0.0000,37.2260,-1.1117,-64.2857
3,2022-12-27,84.26,84.37,83.86,83.86,83.86,754.0,0.0,1.0,-0.474721,...,82.2659,-0.028251,0.019009,0.5542,-0.2399,0.3143,81.1073,74.4519,-0.8960,-71.4286
4,2022-12-23,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,...,82.2535,-0.030825,0.016695,0.6142,-0.2851,0.3291,72.6758,67.7966,-0.7531,-78.5714


In [21]:
# don't need the dates
final_data.drop(columns='index', inplace=True)


In [22]:
final_data = final_data.iloc[1: , :]
final_data.head(60)

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,buy_tomorrow,...,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,macd_hist,macd,stochrsi_fast_k,stochrsi_fast_d,ppo,aroon
1,83.48,84.06,83.12,84.05,84.05,1684.0,0.0,1.0,0.682798,0,...,82.2469,-0.022226,0.021453,0.4478,-0.1845,0.2634,100.0000,68.6130,-1.2474,-35.7143
2,83.50,83.50,83.21,83.28,83.28,1158.0,0.0,1.0,-0.263473,0,...,82.1634,-0.035022,0.013408,0.4940,-0.2410,0.2529,0.0000,37.2260,-1.1117,-64.2857
3,84.26,84.37,83.86,83.86,83.86,754.0,0.0,1.0,-0.474721,0,...,82.2659,-0.028251,0.019009,0.5542,-0.2399,0.3143,81.1073,74.4519,-0.8960,-71.4286
4,83.41,83.72,83.24,83.65,83.65,431.0,0.0,1.0,0.287735,0,...,82.2535,-0.030825,0.016695,0.6142,-0.2851,0.3291,72.6758,67.7966,-0.7531,-78.5714
5,84.20,84.20,83.01,83.29,83.29,443.0,0.0,1.0,-1.080760,0,...,82.2787,-0.035290,0.012142,0.6855,-0.3210,0.3645,44.1339,62.9174,-0.7096,-78.5714
6,83.21,84.20,83.21,84.04,84.04,60.0,0.0,1.0,0.997476,0,...,82.3714,-0.025983,0.019855,0.7657,-0.3247,0.4410,100.0000,81.7008,-0.6703,-92.8571
7,82.17,83.75,82.17,83.75,83.75,742.0,0.0,1.0,1.922843,0,...,82.3569,-0.029506,0.016634,0.8469,-0.3910,0.4559,88.6782,63.4016,-0.7425,-92.8571
8,82.66,83.86,82.66,83.70,83.70,281.0,0.0,1.0,1.258166,1,...,82.3835,-0.030149,0.015729,0.9447,-0.4468,0.4979,67.6404,38.1249,-0.7287,-78.5714
9,83.50,83.50,82.33,82.65,82.65,657.0,0.0,1.0,-1.017964,0,...,82.3590,-0.043291,0.003521,1.0563,-0.5071,0.5492,0.0000,8.6094,-0.7640,-78.5714
10,83.86,83.86,82.80,82.80,82.80,695.0,0.0,1.0,-1.264011,0,...,82.5745,-0.040465,0.002723,1.1831,-0.4694,0.7137,0.0000,17.2188,-0.7512,-71.4286


In [23]:
final_data.describe()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,day_pcent_change,buy_tomorrow,...,bb_lower,pcent_diff_to_bb_upper,pcent_diff_to_bb_lower,macd_signal,macd_hist,macd,stochrsi_fast_k,stochrsi_fast_d,ppo,aroon
count,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000,4.491000e+03,4491.000000,4491.0,4491.000000,4491.000000,...,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000,4491.000000
mean,64.003856,64.672381,63.269501,64.005611,51.955327,1.066142e+05,0.007108,1.0,0.025091,0.154309,...,48.911475,0.169053,0.262175,0.096210,0.000430,0.096640,51.108658,51.106294,0.188003,5.724146
std,21.952144,22.090183,21.765651,21.938207,22.406856,2.337421e+05,0.139798,0.0,1.726053,0.361285,...,21.406853,0.143309,0.127314,0.996443,0.329746,1.062502,40.765792,31.346277,2.387513,63.800152
min,17.500000,17.905000,16.115000,17.400000,11.657600,0.000000e+00,0.000000,1.0,-12.293578,0.000000,...,10.639000,-0.734486,-0.054526,-5.475400,-2.165500,-6.347100,0.000000,0.110700,-15.330200,-100.000000
25%,43.415000,43.825000,42.862500,43.420000,28.253000,2.056950e+04,0.000000,1.0,-0.881415,0.000000,...,26.789400,0.062040,0.156633,-0.392400,-0.175950,-0.421700,2.335900,21.741800,-1.113500,-57.142800
50%,68.611000,69.350000,67.850000,68.860000,57.080000,3.910600e+04,0.000000,1.0,0.040239,0.000000,...,53.003700,0.204050,0.292095,0.113100,0.005600,0.096300,52.198400,51.214800,0.244500,21.428600
75%,82.300000,83.140000,81.432500,82.270000,70.851150,9.635400e+04,0.000000,1.0,0.928003,0.000000,...,67.686600,0.286234,0.370362,0.615900,0.169950,0.646600,100.000000,80.588900,1.696750,64.285700
max,122.890000,123.699000,120.225000,122.998000,99.650000,5.277522e+06,4.000000,1.0,16.092245,1.000000,...,91.049500,0.382821,0.547382,3.382800,1.468500,3.612900,100.000000,99.996200,6.647600,100.000000


In [188]:
train_data = final_data
label = 'buy_tomorrow'
save_path = './autogluon_models'

In [189]:
# there is currently a bug where num_bag_sets >1 produces an error with various algos - waiting on fix
# https://github.com/autogluon/autogluon/issues/2581

predictor = TabularPredictor(label=label, path=save_path, verbosity=3, eval_metric='precision').fit(
    train_data, presets='best_quality', time_limit=15000, num_bag_folds=20, num_stack_levels=3
)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_folds': 20, 'num_stack_levels': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 20,
 'num_bag_sets': None,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 3}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=20, num_bag_sets=20
Saving ./autogluon_models/learner.pkl
Saving ./autogluon_models/predictor.pkl
Beginning AutoGluon training ... Time lim

In [190]:
results = predictor.fit_summary()

Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBM_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/CatBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./autogluon_models/models/XGBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./autogluon

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           CatBoost_BAG_L1   1.000000       0.028020  19.758722                0.028020          19.758722            1       True          7
1           LightGBM_BAG_L2   0.758621       2.482328  24.534868                0.106126          19.512355            2       True         16
2      LightGBMLarge_BAG_L2   0.687500       2.515875  49.400836                0.139673          44.378322            2       True         25
3            XGBoost_BAG_L4   0.681818       4.935526  44.152152                0.387177          28.890719            4       True         47
4         LightGBMXT_BAG_L2   0.655172       2.496622  22.081645                0.120420          17.059132            2       True         15
5         LightGBMXT_BAG_L4   0.655172       4.682258  31.960919                

In [191]:
predictor.leaderboard(train_data, silent=True)

Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBM_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/CatBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./autogluon_models/models/XGBoost_BAG_L1/model.pkl
Loading: ./autogluon_models/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./autogluon_models/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L2/model.pkl
Loading: ./autogluon_models/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./autogluon

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,1.000000,1.000000,0.038095,0.028020,19.758722,0.038095,0.028020,19.758722,1,True,7
1,LightGBM_BAG_L1,1.000000,0.557692,0.214824,0.092479,17.649890,0.214824,0.092479,17.649890,1,True,4
2,RandomForestEntr_BAG_L1,1.000000,0.208791,0.224058,0.256319,2.100045,0.224058,0.256319,2.100045,1,True,6
3,RandomForestGini_BAG_L1,1.000000,0.224490,0.225853,0.266957,1.301742,0.225853,0.266957,1.301742,1,True,5
4,ExtraTreesGini_BAG_L1,1.000000,0.180851,0.235512,0.266096,0.905742,0.235512,0.266096,0.905742,1,True,8
5,ExtraTreesEntr_BAG_L1,1.000000,0.202381,0.236475,0.269624,0.697529,0.236475,0.269624,0.697529,1,True,9
6,LightGBMXT_BAG_L1,1.000000,0.465116,0.304072,0.110265,15.611301,0.304072,0.110265,15.611301,1,True,3
7,XGBoost_BAG_L1,1.000000,0.476923,0.350253,0.389956,31.946066,0.350253,0.389956,31.946066,1,True,11
8,LightGBMLarge_BAG_L1,1.000000,0.491228,0.517237,0.161006,36.715800,0.517237,0.161006,36.715800,1,True,13
9,KNeighborsDist_BAG_L1,1.000000,0.122807,0.605458,0.607851,0.008561,0.605458,0.607851,0.008561,1,True,2


In [192]:
predictor.feature_importance(train_data)

These features in provided data are not utilized by the predictor and will be ignored: ['8. split coefficient']
Loading: ./autogluon_models/models/WeightedEnsemble_L3/model.pkl
Computing feature importance via permutation shuffling for 27 features using 4487 rows with 5 shuffle sets...
Loading: ./autogluon_models/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./autogluon_models/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L1/model.pkl
Loading: ./autogluon_models/models/RandomForestEntr_BAG_L2/model.pkl
Loading: ./autogluon_models/models/RandomForestGini_BAG_L2/model.pkl
Loading: ./autogluon_models/models/WeightedEnsemble_L3/model.pkl
	367.04s	= Expected runtime (73.41s per shuffle set)
Loading: ./autogluon_models/models/ExtraTreesEntr_B

,importance,stddev,p_value,n,p99_high,p99_low
7. dividend amount,0.000000,0.000000,0.500000,5,0.000000,0.000000
day_pcent_change,-0.003655,0.039270,0.577358,5,0.077203,-0.084514
ppo,-0.021273,0.043738,0.831031,5,0.068784,-0.111329
bb_width,-0.071032,0.078455,0.943542,5,0.090507,-0.232571
WMA_20_pcent_diff_to_close,-0.094851,0.030117,0.998928,5,-0.032839,-0.156863
WMA_5_pcent_diff_to_close,-0.103003,0.028470,0.999366,5,-0.044384,-0.161623
stochrsi_fast_k,-0.106143,0.015855,0.999942,5,-0.073498,-0.138789
stochrsi_fast_d,-0.119357,0.058997,0.994686,5,0.002118,-0.240833
WMA_20_pcent_diff_to_open,-0.120010,0.039272,0.998800,5,-0.039148,-0.200871
WMA_10_pcent_diff_to_open,-0.125198,0.022575,0.999878,5,-0.078717,-0.171679
